In [ ]:
#cell 1
!pip install scrapy pandas

In [ ]:
#cell 2
import scrapy
from scrapy.crawler import CrawlerProcess
import pandas as pd
import os

In [ ]:
#cell 3
class MySpider(scrapy.Spider):
    name = "myspider"

    def __init__(self, file_path, *args, **kwargs):
        super(MySpider, self).__init__(*args, **kwargs)
        self.file_path = file_path
        self.output_file = f'/content/{os.path.basename(file_path).replace(".csv", "")}_scrap.csv'

    def start_requests(self):
        # Read the CSV file
        df = pd.read_csv(self.file_path)
        for url in df['Link']:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        # Scraping information from the page
        detail_topic = response.xpath('//detail-topic')
        title = detail_topic.xpath('./@member-topic-title').get(default='Title not found')
        question = detail_topic.xpath('./@member-topic-content').get(default='Question not found')
        question_date = detail_topic.xpath('./@member-post-date').get(default='Question date not found')

        doctor_topic = response.xpath('//doctor-topic')
        answer = doctor_topic.xpath('./@doctor-topic-content').get(default='Answer not found')
        answer_date = doctor_topic.xpath('./@post-date').get(default='Answer date not found')
        by_doctor = doctor_topic.xpath('./@by-doctor').get(default='Doctor not found')

        # Save the results to a dictionary
        yield {
            'URL': response.url,
            'Title': title,
            'Question': question,
            'Question Date': question_date,
            'Answer': answer,
            'Answer Date': answer_date,
            'By Doctor': by_doctor
        }

In [ ]:
#cell 4
# Function to run the Scrapy spider and save the results
def run_spider(file_path):
    process = CrawlerProcess(settings={
        'FEED_FORMAT': 'csv',
        'FEED_URI': f'/content/{os.path.basename(file_path).replace(".csv", "")}_scrap.csv'
    })
    process.crawl(MySpider, file_path=file_path)
    process.start()

In [ ]:
# Running the spider
file_path = '/content/drive/MyDrive/01_komunitastopiklink/trikotilomania(29).csv'
run_spider(file_path)

INFO:scrapy.utils.log:Scrapy 2.11.2 started (bot: scrapybot)
2024-06-30 06:40:03 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.3.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 24.1.0 (OpenSSL 3.2.2 4 Jun 2024), cryptography 42.0.8, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2024-06-30 06:40:03 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.3.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 24.1.0 (OpenSSL 3.2.2 4 Jun 2024), cryptography 42.0.8, Platform Linux-6.1.85+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
2024-06-30 06:40:03 [scrapy.addons] INFO: Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRI

In [ ]:
# # Membaca file CSV ke dalam DataFrame
# scraped_name = f"{os.path.splitext(os.path.basename(file_path))[0]}_scrap.csv"
# df = pd.read_csv(scraped_name)
# df